## Importing of the table from Wikipedia

In [17]:
import urllib.request
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)

In [18]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml") 

In [19]:
soup.title
soup.title.string  #title of the table

'List of postal codes of Canada: M - Wikipedia'

In [20]:
all_tables=soup.find_all("table")  #extracting all the tables from the page

In [21]:
right_table=soup.find('table', class_='wikitable sortable')  #recovering the requested table

## Creating of the dataframe

In [22]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):                  #assigning the values for each cell in the dataframe
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [23]:
import pandas as pd
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

keep_col=["Postal Code", "Borough", "Neighborhood"]
new_df=df[keep_col].replace('\\n', ' ', regex=True)  #removing \n in each data
new_df.columns=['PostalCode', 'Borough', 'Neighborhood']#df_1.set_index('PostalCode', inplace=True)

indexNames=new_df[new_df['Borough']=="Not assigned "].index    #Removing the Borough not assigned
new_df.drop(indexNames , inplace=True)
new_df.reset_index(inplace=True, drop=True)   # reseting the index

keep_col_new=["PostalCode", "Borough", "Neighborhood"]
df_3=new_df[keep_col_new].replace(' / ', ', ', regex=True) #changing '/' into ','

## Preparating of the column 'PostalCode' for merging it with the other table

In [24]:
df_3.PostalCode=df_3.PostalCode.str.strip()
df_3.set_index('PostalCode', inplace=True)
df_3

,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,Business reply mail Processing CentrE


## Importing Geospatial data and creation of the second table

In [25]:
url="http://cocl.us/Geospatial_data"
df_2=pd.read_csv(url)
df_2.columns=['PostalCode', 'Latitude', 'Longitude']

df_2.PostalCode=df_2.PostalCode.str.strip()  #Preparation of the column 'PostalCode' for the merging process

df_2.set_index('PostalCode', inplace=True)
df_2

,Latitude,Longitude
PostalCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
...,...,...
M9N,43.706876,-79.518188
M9P,43.696319,-79.532242
M9R,43.688905,-79.554724


## Merging both tables and preparing the table for the final presentation

In [30]:
final_table=pd.merge(df_3,df_2, right_index=True, left_index=True, how='left')

In [31]:
final_table.reset_index(inplace=True)
final_table

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
